# PROJETO 2 - VISUALIZAÇÃO DADOS (LECD) 2023/2024
##### NOME: Guilherme Cardoso
##### NOME: David Ferreira

### CRIAÇÃO DE DUAS LISTAS PARA FAZER O MERGE

In [ ]:
import pandas as pd

data1 = pd.read_csv('cars.csv') 

aux = data1['Car']
auxlM =[]
auxlO =[]
for i in range(len(aux)):
    a = aux[i]
    a = a.split()
    if a[0] not in auxlM:
        auxlM.append(a[0])
        auxlO.append(data1['Origin'][i])
#print(auxlM)
#print(auxlO)
data1.head(5)

### MERGE DOS DATASETS

In [ ]:
ds = pd.read_csv('Car_sales.csv') 

aa = ds['Manufacturer'].unique()
#print(aa)
lOrigin = []
#print(set(aa) - set(auxlM))

for i in range(len(ds)):
    aux = ds['Manufacturer']
    aux1 = aux[i]
    if aux1 in auxlM:
        indice = auxlM.index(aux1)
        lOrigin.append(auxlO[indice])
    elif aux1 == "Mercedes-B":
        lOrigin.append("Europe")
    elif aux1 == "Jeep":
        lOrigin.append("US")
    elif aux1 == "Hyundai":
        lOrigin.append("South Korea")
    elif aux1 == "Lincoln":
        lOrigin.append("US")
    elif aux1 == "Acura":
        lOrigin.append("Japan")
    elif aux1 == "Saturn":
        lOrigin.append("US")
    elif aux1 == "Mitsubishi":
        lOrigin.append("Japan")
    elif aux1 == "Infiniti":
        lOrigin.append("Japan")
    elif aux1 == "Porsche":
        lOrigin.append("Europe")
    elif aux1 == "Lexus":
        lOrigin.append("Japan")
    elif aux1 == "Jaguar":
        lOrigin.append("Europe")

ds["Origin"] = lOrigin

variaveis = ['Horsepower', 'Curb_weight', 'Fuel_capacity', 'Fuel_efficiency','Price_in_thousands', 'Sales_in_thousands', 'Origin', 'Manufacturer'] 
ds = ds[variaveis]

ds.head(10)

### TRATAMENTO DOS DADOS

##### VALORES EM FALTA

In [ ]:
#ds.info() #ver os valore em falta

ds = ds.dropna(subset=['Horsepower', 'Curb_weight', 'Fuel_efficiency', 'Price_in_thousands'])

ds.info()

### QUANTIDADE DE CARROS POR ZONA

In [ ]:
count_cars = ds.groupby('Origin')['Manufacturer'].size()
print(count_cars)

### QUANTIDADE DE MARCAS POR ZONA 

In [ ]:
count_marcas = ds.groupby('Origin')['Manufacturer'].nunique() 
print(count_marcas)

### MAP PLOT - MARCAS E QUANTIDADE DE CARROS

In [ ]:
import plotly.express as px

cores = {'US': 'blue', 'Japan': 'black', 'South Korea': 'red', 'Europe': 'green'}

coordenadas = {
    'Japan': {'lat': 36.2048, 'lon': 138.2529},
    'US': {'lat': 37.0902, 'lon': -95.7129},
    'Europe': {'lat': 54.5260, 'lon': 15.2551},
    'South Korea': {'lat': 35.9078, 'lon': 127.7669}
}


ds['Latitude'] = ds['Origin'].apply(lambda x: coordenadas[x]['lat'])
ds['Longitude'] = ds['Origin'].apply(lambda x: coordenadas[x]['lon'])

count_cars = ds.groupby('Origin')['Manufacturer'].size().reset_index()
count_cars.columns = ['Origin', 'Carros']
count_marcas = ds.groupby('Origin')['Manufacturer'].nunique().reset_index()
count_marcas.columns = ['Origin', 'Marcas']

count_cars['Latitude'] = count_cars['Origin'].apply(lambda x: coordenadas[x]['lat'])
count_cars['Longitude'] = count_cars['Origin'].apply(lambda x: coordenadas[x]['lon'])
count_cars['Cor'] = count_cars['Origin'].apply(lambda x: cores[x])
cores = {'US': 'blue', 'Japan': 'black', 'South Korea': 'red', 'Europe': 'green'}

count_cars = count_cars.merge(count_marcas, on='Origin')

fig = px.scatter_geo(count_cars,
                     lat = 'Latitude',
                     lon = 'Longitude',
                     size = 'Carros',
                     color = 'Origin',  
                     color_discrete_map = cores,  
                     hover_name = 'Origin',
                     hover_data = {'Carros': True, 'Marcas': True,'Latitude': False, 'Longitude': False, 'Cor': False, 'Origin': False},
                     title = 'Quantidade de Carros e Marcas por Origem',
                     projection = 'natural earth')
fig.show()


### BAR CHART -- PREÇOS ORIGEM E MARCA DOS CARROS

In [ ]:
import plotly.graph_objects as go

colorsOrigem = ['black','green','blue','red']
colorsMarca = []

marcas = ds['Manufacturer'].unique()
marcasCOrigem = ds.groupby('Origin')['Manufacturer'].unique()

for i in marcas:
    for j in range(len(marcasCOrigem)):
        if i in marcasCOrigem[j]:
            if j == 0:
                colorsMarca.append(colorsOrigem[1])
            elif j == 1:
                colorsMarca.append(colorsOrigem[0])
            elif j == 2:
                colorsMarca.append(colorsOrigem[3])
            elif j == 3:
                colorsMarca.append(colorsOrigem[2])


precoPorOrigem = ds.groupby('Origin')['Price_in_thousands'].mean()
precoPorOrigem = (precoPorOrigem * 1000).round(2)
#print(precoPorOrigem)

precoPorMarca = ds.groupby('Manufacturer')['Price_in_thousands'].mean()
precoPorMarca = (precoPorMarca * 1000).round(2)
#print(precoPorMarca)

def bargraph(preco,base,cor,v):
    fig = go.Figure(data=[
        go.Bar(x=base, y=preco,marker=dict(color=cor))
    ])

    fig.update_layout(
        title='Preço Médio por '+str(v)+' de Carro',
        xaxis=dict(title=v),
        yaxis=dict(title='Preço Médio')
    )

    fig.show()

bargraph(precoPorOrigem,ds['Origin'].unique(),colorsOrigem,'Origem')
bargraph(precoPorMarca,ds['Manufacturer'].unique(),colorsMarca,'Marca') 

### SCATTER PLOT -- PESO vs CAVALAGEM // CAPACIDADE vs EFICIÊNCIA 

In [ ]:
colorsPC = []
colorsCE = []

def definirCores(ds, lc):
    cores = []
    for origin in ds['Origin']:
        if origin == 'Europe':
            cores.append(lc[1])
        elif origin == 'Japan':
            cores.append(lc[0])
        elif origin == 'South Korea':
            cores.append(lc[3])
        elif origin == 'US':
            cores.append(lc[2])
    return cores

colorsPC = definirCores(ds, colorsOrigem)
colorsCE = definirCores(ds, colorsOrigem)



In [ ]:
def scattergraph(x,y,xname,yname,titulo,cor):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='markers',
        marker=dict(color=cor,size=7)
    ))
    fig.update_layout(
            title=titulo,
            xaxis=dict(title=xname),
            yaxis=dict(title=yname) 
        )
    fig.show()

scattergraph(ds['Curb_weight'],ds['Horsepower'],'Peso','Cavalos','Peso/Cavalos',colorsPC)
scattergraph(ds['Fuel_capacity'],ds['Fuel_efficiency'],'Capacidade','Eficiencia','Capacidade/Eficiencia',colorsCE)


### BAR CHART - VENDAS POR MARCA

In [ ]:
vendasPorFabricante = ds.groupby('Manufacturer')['Sales_in_thousands'].sum()
vendasPorFabricante = vendasPorFabricante.reset_index()


colorsOrigem = ['black','green','blue','red']
colorsMarca = []

marcas = ds['Manufacturer'].unique()
marcasCOrigem = ds.groupby('Origin')['Manufacturer'].unique()

for i in marcas:
    for j in range(len(marcasCOrigem)):
        if i in marcasCOrigem[j]:
            if j == 0:
                colorsMarca.append(colorsOrigem[1])
            elif j == 1:
                colorsMarca.append(colorsOrigem[0])
            elif j == 2:
                colorsMarca.append(colorsOrigem[3])
            elif j == 3:
                colorsMarca.append(colorsOrigem[2])

def bargraph(vendas, fabricantes, cores):
    fig = go.Figure(data=[
        go.Bar(x=fabricantes, y=vendas, marker=dict(color=cores))
    ])

    fig.update_layout(
        title='Vendas Totais por Fabricante',
        xaxis=dict(title='Fabricante'),
        yaxis=dict(title='Vendas Totais')
    )

    fig.show()

bargraph(vendasPorFabricante['Sales_in_thousands'], vendasPorFabricante['Manufacturer'], colorsMarca)
